In [1]:
import numpy as np
import pandas as pd

In [2]:
# load datasets
datasets = {}
years = range(2016, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/3686568105.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/3686568105.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/3686568105.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/3686568105.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2022 Loading...
success


In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]

In [4]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if col.startswith('eo')])

['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']


In [42]:
schools_edrpou = pd.DataFrame(columns=['eoname', 'eoregname', 'eoareaname', 'eotername'])

In [43]:
for year, dataset in datasets.items():
    schools_edrpou = pd.concat([schools_edrpou, dataset.loc[:, ['eoname', 'eoregname', 'eoareaname', 'eotername']]], ignore_index=True)

schools_edrpou

,eoname,eoregname,eoareaname,eotername
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів
2,NaN,NaN,NaN,NaN
3,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста
4,NaN,NaN,NaN,NaN
...,...,...,...,...
2201113,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці
2201114,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці
2201115,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці
2201116,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці


In [44]:
schools_edrpou.drop_duplicates(inplace=True)
# when paricipant graduated in the previous years the corresponding information about school
# (everything with 'eo') is NaN
#we drop these rows
schools_edrpou.dropna(how = 'all', inplace=True)
schools_edrpou.reset_index(drop=True, inplace=True)
schools_edrpou

,eoname,eoregname,eoareaname,eotername
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів
2,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,Донецька область,Донецька область,м.Дружківка
4,Тернопільська спеціалізована школа І-ІІІ ступе...,Тернопільська область,Тернопільська область,м.Тернопіль
...,...,...,...,...
34380,Гульський ліцей Стриївської сільської ради Нов...,Житомирська область,Новоград-Волинський район,с.Гульськ
34381,Киківський ліцей Стриївської сільської ради Но...,Житомирська область,Новоград-Волинський район,с.Кикова
34382,Литовезький ліцей імені Володимира Якобчука Во...,Волинська область,Володимирський район,с.Литовеж
34383,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці


In [45]:
schools_edrpou[schools_edrpou.eoname == 'Лучицький навчально-виховний комплекс "Загальноосвітня школа І-ІІІ ступенів - дитячий садок" Сокальської районної ради Львівської області\r\n']

,eoname,eoregname,eoareaname,eotername
8484,"Лучицький навчально-виховний комплекс ""Загальн...",Львівська область,Львівська область,Сокальський район


In [47]:
locations = pd.read_csv('./location_data/locations.csv', encoding='utf-8', dtype=str)
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
28878,Інші країни,м.Гронінген,м.Гронінген,0017010000,OC17010000000000000
28879,Інші країни,м.Люксембург,м.Люксембург,0015010000,OC15010000000000000
28880,Інші країни,м.Марібор,м.Марібор,0023010000,OC23010000000000000
28881,Інші країни,м.Сакраменто,м.Сакраменто,0024020000,OC24020000000000000


In [48]:
schools_edrpou = schools_edrpou.merge(locations[['KOATUU_2020']], left_on=[schools_edrpou['eoregname'],schools_edrpou['eoareaname'], schools_edrpou['eotername']], right_on=[locations['regname'],locations['areaname'], locations['tername']], how='left')
schools_edrpou = schools_edrpou.drop(columns = ['key_0', 'key_1', 'key_2'], axis=1)
schools_edrpou.head()

,eoname,eoregname,eoareaname,eotername,KOATUU_2020
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння,2323085101
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів,6822710100
2,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,Донецька область,Донецька область,м.Дружківка,1411700000
4,Тернопільська спеціалізована школа І-ІІІ ступе...,Тернопільська область,Тернопільська область,м.Тернопіль,6110100000


In [50]:
schools_edrpou

,eoname,eoregname,eoareaname,eotername,KOATUU_2020
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння,2323085101
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів,6822710100
2,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,Донецька область,Донецька область,м.Дружківка,1411700000
4,Тернопільська спеціалізована школа І-ІІІ ступе...,Тернопільська область,Тернопільська область,м.Тернопіль,6110100000
...,...,...,...,...,...
34450,Гульський ліцей Стриївської сільської ради Нов...,Житомирська область,Новоград-Волинський район,с.Гульськ,1824081001
34451,Киківський ліцей Стриївської сільської ради Но...,Житомирська область,Новоград-Волинський район,с.Кикова,1824081801
34452,Литовезький ліцей імені Володимира Якобчука Во...,Волинська область,Володимирський район,с.Литовеж,0721182401
34453,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,0721182403


In [51]:
schools_edrpou = schools_edrpou[schools_edrpou.KOATUU_2020.notna()]

In [66]:
schools_edrpou_id= schools_edrpou[['eoname', 'KOATUU_2020']]
schools_edrpou_id.drop_duplicates(inplace=True)
schools_edrpou_id.reset_index(drop=True, inplace=True)
schools_edrpou_id

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/3377070480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_edrpou_id.drop_duplicates(inplace=True)


,eoname,KOATUU_2020
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101
1,Красилівська загальноосвітня школа I-III ступе...,6822710100
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000
...,...,...
27505,Гульський ліцей Стриївської сільської ради Нов...,1824081001
27506,Киківський ліцей Стриївської сільської ради Но...,1824081801
27507,Литовезький ліцей імені Володимира Якобчука Во...,0721182401
27508,Заболотцівський ліцей Литовезької сільської ра...,0721182403


In [67]:
def create_id(dataset, attr):
    dataset.loc[:,'id'] = dataset[attr]

    abbr = {'КЗ ':'Комунальний заклад ', 'ОНЗ ':'Опорний навчальний заклад ', 'ОЗЗСО ':'Опорний заклад загальної середньої освіти ',
            'ЗЗСО ':'Заклад загальної середньої освіти ', 'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
            'НВК ':'Навчально-виховний комплекс ', 'ЗОШ ':'Загальнооосвітня школа ', 'СЗШ ':'Середня загальноосвітня школа ', 'ТОВ ': 'Товариство з обмеженою відповідальністю '}

    for ab in abbr:
        dataset.loc[:,'id'] = dataset['id'].str.replace(ab, abbr[ab])

    dataset.loc[:,'id'] = dataset['id'].str.replace('[\s]', '', regex = True)
    dataset.loc[:,'id'] = dataset['id'].str.replace('[\r]', '', regex = True)
    dataset.loc[:,'id'] = dataset['id'].str.replace('[\n]', '', regex = True)
    dataset.loc[dataset['id'].str.endswith('обл'), 'id'] = dataset.loc[dataset['id'].str.endswith('обл'), 'id'] + '.'
    dataset.loc[:,'id'] = dataset['id'].str.replace('І', 'I')
    dataset.loc[:,'id'] = dataset['id'].str.replace('I-III', '3')
    dataset.loc[:,'id'] = dataset['id'].str.replace('I-II', '2')
    dataset.loc[:,'id'] = dataset['id'].str.lower()

    dct = {'с.':'села', 'ст.':'ступенів', 'ім.':'імені', 'iм.':'імені', 'обл.': 'області', 
        '||||':'3', '|||':'2',
        'i':'', 'і':'', 'ї':'', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'г', 'e':'е', 'a':'а', 'k':'к', 'x': 'х', 
        "'":'', "’":'', ",":'', ".":'', "«":'', "»":'', "–":'', "-":'', "*":"",
        "_x000d_":'', '_x000D_':'', "_":'', "(":'', ")":'', "[":'', "]":'', "}":'', "{":'', 
        "нвк":'навчальновиховнийкомплекс', 'знз': 'загальноосвтнйнавчальнийзаклад', 'зош': 'загальноосвтняшкола', 
        'сзш':'середнязагальноосвтняшкола', 'нн':'н'}

    for symb in dct:
        dataset.loc[:,'id'] = dataset['id'].str.replace(symb, dct[symb])

    dataset.loc[:,'id'] = dataset['id'].str.replace('"', '', regex = True)

create_id(schools_edrpou_id, 'eoname')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_81153/98297597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,'id'] = dataset[attr]


In [68]:
schools_edrpou_id

,eoname,KOATUU_2020,id
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...
...,...,...,...
27505,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...
27506,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...
27507,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...
27508,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...


# MON data 2023

In [56]:
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [64]:
schools_data_id = schools_data

In [65]:
create_id(schools_data_id, 'Повна назва')
schools_data_id

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш№2мкиввдокремленийстру...
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя№236мстакиєва
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя№87мстакиєва
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя№150мстакиєва
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя№179мстакиєва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступенвборзнянсь...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступенвсеменвськогмназ№2семенвсь...


In [69]:
schools_edrpou_id

,eoname,KOATUU_2020,id
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...
...,...,...,...
27505,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...
27506,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...
27507,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...
27508,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...


In [70]:
schools_ed = schools_edrpou_id.merge(schools_data_id[['Код ЄДРПОУ']], left_on=[schools_edrpou_id['id'],schools_edrpou_id['KOATUU_2020']], right_on=[schools_data_id['id'],schools_data_id['Код КОАТУУ']], how='left')
schools_ed = schools_ed.drop(columns = ['key_0', 'key_1'], axis=1)
schools_ed.head()

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,NaN
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,NaN
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,NaN
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173


In [71]:
schools_ed[schools_ed['Код ЄДРПОУ'].notna()]

,eoname,KOATUU_2020,id,Код ЄДРПОУ
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
38,Слов'янський педагогічний ліцей Слов'янської м...,1414100000,словянськийпедагогчнийлцейсловянськомськорадид...,33534376
46,Комунальний заклад «Харківська загальноосвітня...,6310138500,комунальнийзакладхарквськазагальноосвтняшкола3...,25779605
47,Черкаська загальноосвітня школа І-ІІІ ступенів...,7110136400,черкаськазагальноосвтняшкола3ступенв№15черкась...,25660387
...,...,...,...,...
27509,Свірзький ліцей Бібрської міської ради Львівсь...,4623386401,сврзькийлцейббрськомськорадильввськогорайонуль...,22368930
27510,Суслівський ліцей Стриївської сільської ради Н...,1824085601,суслвськийлцейстривськосльськорадиновоградволи...,06670121
27511,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27512,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773


# Finance 2019

In [72]:
schools_ed.loc[schools_ed['Код ЄДРПОУ'].isna(), 'Код ЄДРПОУ'] = ''

In [73]:
schools_ed

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27511,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27512,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773
27513,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,
27514,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,


In [74]:
finance_data_2019 =  pd.read_excel('/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/NY fairness/NeurIPS/ZNO-Dataset/notebooks/school_level/fin_data/ZZSO-2019.xlsx', header = 1, dtype='str')
finance_data_2019

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка
0,2,Вінницька,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,9915.2,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN
1,2,Вінницька,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13692.2,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN
2,2,Вінницька,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,16184.7,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN
3,2,Вінницька,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14321.9,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN
4,2,Вінницька,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,15061,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,місто Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,6642.512,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN
14171,26,місто Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,10285.5,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN
14172,26,місто Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,14547.4,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN
14173,26,місто Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,6973.8,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN


In [75]:
create_id(finance_data_2019, 'Найменування закладу')

In [77]:
finance_data_2019.columns

Index(['№ Області', 'Область', 'Тип школи: міська / сільська', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
       'Кількість учнів разом',
       'Разом кількість учнів, та дітей, які здобувають дошкільну освіту',
       'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на функціонування закладу 

In [78]:
finance_data_2019.rename(columns={'ЄДРПОУ':'Код ЄДРПОУ'}, inplace=True)

In [79]:
finance_data_2019

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,Код ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN,загальноосвтняшкола3ступенв№1комунальновласнос...
1,2,Вінницька,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,2,Вінницька,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN,загальноосвтняшкола3ступенв№3комунальновласнос...
3,2,Вінницька,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN,загальноосвтняшкола3ступенв№4комунальновластно...
4,2,Вінницька,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,місто Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN,школаступеня№326
14171,26,місто Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN,школаступеня№333дарницькогорайонумкиєва
14172,26,місто Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN,початковашкола№332дарницькогорайонумкиєва
14173,26,місто Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN,початковашкола№334дарницькогорайонумкиєва


In [80]:
schools_ed[schools_ed['Код ЄДРПОУ'] != '']

,eoname,KOATUU_2020,id,Код ЄДРПОУ
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
38,Слов'янський педагогічний ліцей Слов'янської м...,1414100000,словянськийпедагогчнийлцейсловянськомськорадид...,33534376
46,Комунальний заклад «Харківська загальноосвітня...,6310138500,комунальнийзакладхарквськазагальноосвтняшкола3...,25779605
47,Черкаська загальноосвітня школа І-ІІІ ступенів...,7110136400,черкаськазагальноосвтняшкола3ступенв№15черкась...,25660387
...,...,...,...,...
27509,Свірзький ліцей Бібрської міської ради Львівсь...,4623386401,сврзькийлцейббрськомськорадильввськогорайонуль...,22368930
27510,Суслівський ліцей Стриївської сільської ради Н...,1824085601,суслвськийлцейстривськосльськорадиновоградволи...,06670121
27511,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27512,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773


In [81]:
schools_ed_fin = schools_ed.merge(finance_data_2019[['Код ЄДРПОУ']], left_on=[schools_ed['id']], right_on=[finance_data_2019['id']], how='left')
schools_ed_fin = schools_ed_fin.drop(columns = ['key_0'], axis=1)
schools_ed_fin.head()

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173


In [83]:
schools_ed_fin.columns

Index(['eoname', 'KOATUU_2020', 'id', 'Код ЄДРПОУ_x', 'Код ЄДРПОУ_y'], dtype='object')

In [84]:
schools_ed_fin[schools_ed_fin['Код ЄДРПОУ_y'].notna()]

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27314,Опорний заклад Почаївська загальноосвітня школ...,6123410500,опорнийзакладпочавськазагальноосвтняшкола3ступенв,,23591156
27377,"Навчально-виховний комплекс ""загальноосвітня ш...",0724283001,навчальновиховнийкомплексзагальноосвтняшкола3с...,21747861,21747861
27431,Спеціалізована школа № 254 Святошинського райо...,8038600000,спецалзованашкола№254святошинськогорайонумкиєва,22878619,22878619
27443,Криворізька загальноосвітня школа І-ІІІ ступен...,1211037000,криворзьказагальноосвтняшкола3ступенв№124криво...,,33355806


In [85]:
schools_ed_fin.loc[schools_ed_fin['Код ЄДРПОУ_y'].isna(), 'Код ЄДРПОУ_y'] = ''

In [86]:
schools_ed_fin

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27521,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27522,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [87]:
schools_ed_fin.loc[schools_ed_fin['Код ЄДРПОУ_x'] == '', 'Код ЄДРПОУ_x'] = schools_ed_fin.loc[schools_ed_fin['Код ЄДРПОУ_x'] == '', 'Код ЄДРПОУ_y']
schools_ed_fin

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27521,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27522,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [88]:
schools_ed_fin = schools_ed_fin.drop(columns=['Код ЄДРПОУ_y'], axis=1)
schools_ed_fin = schools_ed_fin.rename(columns = {'Код ЄДРПОУ_x':'Код ЄДРПОУ'})
schools_ed_fin

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773
27521,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,
27522,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,


In [89]:
schools_ed_fin[schools_ed_fin['Код ЄДРПОУ'] != '']

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27517,Свірзький ліцей Бібрської міської ради Львівсь...,4623386401,сврзькийлцейббрськомськорадильввськогорайонуль...,22368930
27518,Суслівський ліцей Стриївської сільської ради Н...,1824085601,суслвськийлцейстривськосльськорадиновоградволи...,06670121
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773


# Finance 2020

In [91]:
schools_ed_fin_new =schools_ed_fin
schools_ed_fin_new

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773
27521,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,
27522,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,


In [92]:
finance_data_2020 =  pd.read_excel('/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/NY fairness/NeurIPS/ZNO-Dataset/notebooks/school_level/fin_data/ZZSO-2020.xlsx', header = 1, dtype='str')
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,4388.111,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,6523.777,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,9273.777,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,8654.501,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,14833.298,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,5415.94732,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,9401.191,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,9552.78,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,11503.71,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN


In [93]:
create_id(finance_data_2020, 'Найменування закладу')

In [95]:
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступен...
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступенв№3козятинськомсь...
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступенв№5козятинськ...
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола№334дарницькогорайонумкиєва
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступеня№333дарницькогорайонумкиєва


In [94]:
finance_data_2020.columns

Index(['№ Області', 'Область', 'Тип школи: міська / сільська', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %',
       'Кількість учнів разом', 'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)',
       'Видатки на функціонування 

In [96]:
finance_data_2020.rename(columns={'ЄДРПОУ':'Код ЄДРПОУ'}, inplace=True)

In [97]:
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,Код ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступен...
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступенв№3козятинськомсь...
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступенв№5козятинськ...
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола№334дарницькогорайонумкиєва
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступеня№333дарницькогорайонумкиєва


In [98]:
schools_ed_fin_new[schools_ed_fin_new['Код ЄДРПОУ'] != '']

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27517,Свірзький ліцей Бібрської міської ради Львівсь...,4623386401,сврзькийлцейббрськомськорадильввськогорайонуль...,22368930
27518,Суслівський ліцей Стриївської сільської ради Н...,1824085601,суслвськийлцейстривськосльськорадиновоградволи...,06670121
27519,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27520,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773


In [99]:
schools_ed_fin_new2 = schools_ed_fin_new.merge(finance_data_2020[['Код ЄДРПОУ']], left_on=[schools_ed_fin_new['id']], right_on=[finance_data_2020['id']], how='left')
schools_ed_fin_new2 = schools_ed_fin_new2.drop(columns = ['key_0'], axis=1)


,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173


In [100]:
schools_ed_fin_new2.head()

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173


In [101]:
schools_ed_fin_new2[schools_ed_fin_new2['Код ЄДРПОУ_y'].notna()]

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27324,Опорний заклад Почаївська загальноосвітня школ...,6123410500,опорнийзакладпочавськазагальноосвтняшкола3ступенв,23591156,23591156
27387,"Навчально-виховний комплекс ""загальноосвітня ш...",0724283001,навчальновиховнийкомплексзагальноосвтняшкола3с...,21747861,21747861
27399,Новосільська загальноосвітня школа І-ІІІ ступе...,6124685001,новосльськазагальноосвтняшкола3ступенвменмирон...,24619612,24619612
27453,Криворізька загальноосвітня школа І-ІІІ ступен...,1211037000,криворзьказагальноосвтняшкола3ступенв№124криво...,33355806,33355806


In [102]:
schools_ed_fin_new2.loc[schools_ed_fin_new2['Код ЄДРПОУ_y'].isna(), 'Код ЄДРПОУ_y'] = ''

In [103]:
schools_ed_fin_new2

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [104]:
schools_ed_fin_new2.loc[schools_ed_fin_new2['Код ЄДРПОУ_x'] == '', 'Код ЄДРПОУ_x'] = schools_ed_fin_new2.loc[schools_ed_fin_new2['Код ЄДРПОУ_x'] == '', 'Код ЄДРПОУ_y']
schools_ed_fin_new2

,eoname,KOATUU_2020,id,Код ЄДРПОУ_x,Код ЄДРПОУ_y
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,14040173
...,...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [105]:
schools_ed_fin_new2 = schools_ed_fin_new2.drop(columns=['Код ЄДРПОУ_y'], axis=1)
schools_ed_fin_new2 = schools_ed_fin_new2.rename(columns = {'Код ЄДРПОУ_x':'Код ЄДРПОУ'})
schools_ed_fin_new2

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,


In [106]:
schools_ed_fin_new2[schools_ed_fin_new2['Код ЄДРПОУ'] != '']

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27527,Свірзький ліцей Бібрської міської ради Львівсь...,4623386401,сврзькийлцейббрськомськорадильввськогорайонуль...,22368930
27528,Суслівський ліцей Стриївської сільської ради Н...,1824085601,суслвськийлцейстривськосльськорадиновоградволи...,06670121
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773


# Universities

In [109]:
univ_data = pd.read_excel('./school_data/zakladi-vishchoyi-osviti-09.xlsx', dtype = str)
univ_data

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Поштовий індекс (юридична),Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника"
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,03680,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,03115,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,04071,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,03039,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,03067,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,17400,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,16400,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,17000,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,17044,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна


In [110]:
univ_data_id = univ_data

In [111]:
create_id(univ_data_id, 'Назва закладу освіти')
univ_data_id

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризму
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладує...
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийсльськогосподарськийтехнкум
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарномедициниблоцерквсь...
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайну


In [112]:
univ_data_id.columns

Index(['Назва закладу освіти', 'Код', 'Код головного закладу', 'Коротка назва',
       'Назва закладу освіти (англ.)', 'Рік заснування',
       'Ідентифікаційний код', 'Вид закладу освіти', 'Тип закладу освіти',
       'Форма власності',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Поштовий індекс (фактична)', 'Код КОАТУУ (фактична)',
       'Регіон (фактична)', 'Населений пункт (фактична)', 'Адреса (фактична)',
       'Поштовий індекс (юридична)', 'Код КОАТУУ (юридична)',
       'Регіон (юридична)', 'Населений пункт (юридична)', 'Адреса (юридична)',
       'Телефон / факс', 'Електронна пошта', 'Веб-сайт', 'Посада керівника',
       'Прізвище, ім'я, по батькові керівника', 'id'],
      dtype='object')

In [113]:
schools_ed_univ = schools_ed_fin_new2.merge(univ_data_id[['Ідентифікаційний код']], left_on=[schools_ed_fin_new2['id'],schools_ed_fin_new2['KOATUU_2020']], right_on=[univ_data_id['id'],univ_data_id['Код КОАТУУ (фактична)']], how='left')
schools_ed_univ = schools_ed_univ.drop(columns = ['key_0', 'key_1'], axis=1)
schools_ed_univ.head()

,eoname,KOATUU_2020,id,Код ЄДРПОУ,Ідентифікаційний код
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,NaN
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,NaN
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,NaN
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,NaN
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,NaN


In [115]:
schools_ed_univ[schools_ed_univ['Ідентифікаційний код'].notna()]

,eoname,KOATUU_2020,id,Код ЄДРПОУ,Ідентифікаційний код
27,Харківське обласне вище училище фізичної культ...,6310136600,харквськеобласневищеучилищефзичнокультуриспорту,,05401005
1504,Комунальний вищий навчальний заклад Київської ...,3210600000,комунальнийвищийнавчальнийзакладкивськообласно...,,22208066
10443,Хмельницький базовий медичний коледж,6810100000,хмельницькийбазовиймедичнийколедж,,02010965
12926,Донецьке вище училище олімпійського резерву ім...,1410300000,донецькевищеучилищеолмпйськогорезервуменселабубки,,20361881
13188,Технолого-економічний коледж Білоцерківського ...,3210300000,технологоекономчнийколеджблоцерквськогонацонал...,,00419680
...,...,...,...,...,...
19664,"Дубенський коледж вищого навчального закладу ""...",5610300000,дубенськийколеджвищогонавчальногозакладувдкрит...,,38537717
19882,Відокремлений структурний підрозділ «Костянтин...,1422482701,вдокремленийструктурнийпдроздлкостянтинвськийк...,,34641716
20247,Волинський коледж Приватного вищого навчальног...,0710100000,волинськийколеджприватноговищогонавчальногозак...,,38690505
20441,Запорізький авіаційний коледж імені О. Г. Івченка,2310137500,запорзькийавацйнийколеджменогвченка,,20507735


In [116]:
schools_ed_univ.loc[schools_ed_univ['Ідентифікаційний код'].isna(), 'Ідентифікаційний код'] = ''

In [117]:
schools_ed_univ

,eoname,KOATUU_2020,id,Код ЄДРПОУ,Ідентифікаційний код
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,
...,...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [118]:
schools_ed_univ.loc[schools_ed_univ['Код ЄДРПОУ'] == '', 'Код ЄДРПОУ'] = schools_ed_univ.loc[schools_ed_univ['Код ЄДРПОУ'] == '', 'Ідентифікаційний код']
schools_ed_univ

,eoname,KOATUU_2020,id,Код ЄДРПОУ,Ідентифікаційний код
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098,
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114,
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046,
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061,
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173,
...,...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744,
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773,
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,,


In [119]:
schools_ed_univ = schools_ed_univ.drop(columns=['Ідентифікаційний код'], axis=1)
schools_ed_univ

,eoname,KOATUU_2020,id,Код ЄДРПОУ
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",2323085101,терпнвськийколегумджереломелтопольськорайонора...,26373098
1,Красилівська загальноосвітня школа I-III ступе...,6822710100,красилвськазагальноосвтняшкола3ступенв№3красил...,25880114
2,Чернівецька спеціалізована школа І-ІІІ ступені...,7310100000,чернвецькаспецалзованашкола3ступенвфзикоматема...,21431046
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,1411700000,загальноосвтняшкола3ступенв№6дружквськомськора...,25705061
4,Тернопільська спеціалізована школа І-ІІІ ступе...,6110100000,тернопльськаспецалзованашкола3ступенв№3зпоглиб...,14040173
...,...,...,...,...
27529,Гульський ліцей Стриївської сільської ради Нов...,1824081001,гульськийлцейстривськосльськорадиновоградволин...,06669744
27530,Киківський ліцей Стриївської сільської ради Но...,1824081801,киквськийлцейстривськосльськорадиновоградволин...,06669773
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,
27532,Заболотцівський ліцей Литовезької сільської ра...,0721182403,заболотцвськийлцейлитовезькосльськорадиволодим...,


In [122]:
schools_ed_univ[schools_ed_univ['Код ЄДРПОУ'] == '']

,eoname,KOATUU_2020,id,Код ЄДРПОУ
5,"Тарасівський навчально-виховний комплекс ""Зага...",3523486901,тарасвськийнавчальновиховнийкомплексзагальноос...,
8,"Великопільський навчально-виховний комплекс ""з...",4625881601,великопльськийнавчальновиховнийкомплексзагальн...,
9,Лоївська загальноосвітня школа І-ІІІ ступенів ...,2624083401,ловськазагальноосвтняшкола3ступенвнадврнянсько...,
10,"Хоровецький навчально- виховний комплекс ""дошк...",6823987719,хоровецькийнавчальновиховнийкомплексдошкльнийн...,
11,"Кілійський навчально-виховний комплекс ""Кілійс...",5122310100,клйськийнавчальновиховнийкомплексклйськазагаль...,
...,...,...,...,...
27515,Зорянський заклад загальної середньої освіти і...,1422482708,зорянськийзакладзагальносередньоосвтименгерояр...,
27522,Стратіївський заклад загальної середньої освіт...,0525085301,стратвськийзакладзагальносередньоосвтиольгопль...,
27525,Бібрський опорний ліцей імені Уляни Кравченко ...,4623310300,ббрськийопорнийлцейменуляникравченкоббрськомсь...,
27531,Литовезький ліцей імені Володимира Якобчука Во...,0721182401,литовезькийлцейменволодимираякобчукаволодимирв...,


In [121]:
schools_ed_univ.to_csv('schools_edrpou.csv', index = False)